In [51]:
import polars as pl
import numpy as np
from pathlib import Path
import plotly.express as px
project_dir = Path(globals()['_dh'][0]).parent
import json
from aw_client.client import ActivityWatchClient
from datetime import datetime, timedelta,timezone
from typing import List,Tuple



First, let's examine two intervals of time, the first a day where I was entirely unfocused and had little capacity or incentive to work on anything productive

Second, a day where I spent a good portion of my time either coding, or examining resources related to whatever I was working on

In [52]:
from pprint import pprint
from aw_core import Event
hostname='Dominion'
client=ActivityWatchClient(hostname)



aw_window_events=client.get_events(f'aw-watcher-window_{hostname}')
aw_afk_events=client.get_events(f'aw-watcher-afk_{hostname}')

def get_bounds(client: ActivityWatchClient,start: datetime,end: datetime) -> Tuple[datetime, datetime]:
    afk_events=client.get_events(f'aw-watcher-afk_{hostname}',start=start,end=end)
    
    new_start=start
    new_end=end
    for event in afk_events[::-1]:
        print(event)
        if event.data['status']=='not-afk':
            
            new_start=event.timestamp
            break
    for event in afk_events:
        if event.data['status']=='not-afk':
            new_end=event.timestamp
            break
    return new_start,new_end
    

unfocused_start=datetime.fromisoformat('2023-04-29T23:00:00').astimezone(tz=timezone(timedelta(0)))
unfocused_end=datetime.fromisoformat('2023-04-30T10:00:00').astimezone(tz=timezone(timedelta(0)))

flow_start=datetime.fromisoformat('2023-04-26T23:00:00').astimezone(tz=timezone(timedelta(0)))
flow_end=datetime.fromisoformat('2023-04-27T10:00:00').astimezone(tz=timezone(timedelta(0)))
pprint(client.get_events(f'aw-watcher-afk_{hostname}',start=flow_start,end=flow_end))
unfocused_start,unfocused_end=get_bounds(client,unfocused_start,unfocused_end)
flow_start,flow_end=get_bounds(client,flow_start,flow_end)
print(flow_start.astimezone())
unfocused_events=client.get_events(f'aw-watcher-window_{hostname}',start=unfocused_start,end=unfocused_end)
flow_events=client.get_events(f'aw-watcher-window_{hostname}',start=flow_start,end=flow_end)


[{'data': {'status': 'not-afk'},
  'duration': datetime.timedelta(seconds=567, microseconds=170232),
  'id': 46997,
  'timestamp': datetime.datetime(2023, 4, 27, 14, 50, 32, 829000, tzinfo=datetime.timezone.utc)},
 {'data': {'status': 'afk'},
  'duration': datetime.timedelta(seconds=54, microseconds=156187),
  'id': 46996,
  'timestamp': datetime.datetime(2023, 4, 27, 14, 49, 34, 334000, tzinfo=datetime.timezone.utc)},
 {'data': {'status': 'not-afk'},
  'duration': datetime.timedelta(seconds=10, microseconds=268224),
  'id': 46992,
  'timestamp': datetime.datetime(2023, 4, 27, 14, 47, 23, 222000, tzinfo=datetime.timezone.utc)},
 {'data': {'status': 'afk'},
  'duration': datetime.timedelta(seconds=34, microseconds=614439),
  'id': 46991,
  'timestamp': datetime.datetime(2023, 4, 27, 14, 46, 43, 876000, tzinfo=datetime.timezone.utc)},
 {'data': {'status': 'not-afk'},
  'duration': datetime.timedelta(seconds=4772, microseconds=189252),
  'id': 46549,
  'timestamp': datetime.datetime(2023,

In [54]:
afk_1=client.get_event(f"aw-watcher-afk_{hostname}",event_id=46476)
afk_2= client.get_event(f"aw-watcher-afk_{hostname}",event_id=46478)
noise_event= client.get_event(f'aw-watcher-window_{hostname}',event_id=46477)
print(f"afk event 1: {afk_1}\nNoise event: {noise_event}\nafk event 2: {afk_2}")

afk event 1: {'id': 46476, 'timestamp': datetime.datetime(2023, 4, 27, 10, 26, 48, 492000, tzinfo=datetime.timezone.utc), 'duration': datetime.timedelta(seconds=4005, microseconds=756975), 'data': {'status': 'afk'}}
Noise event: {'id': 46477, 'timestamp': datetime.datetime(2023, 4, 27, 11, 33, 50, 387000, tzinfo=datetime.timezone.utc), 'duration': datetime.timedelta(seconds=4165, microseconds=290022), 'data': {'app': 'Ripcord', 'title': 'Slack Authorization Failed'}}
afk event 2: {'id': 46478, 'timestamp': datetime.datetime(2023, 4, 27, 11, 33, 50, 398000, tzinfo=datetime.timezone.utc), 'duration': datetime.timedelta(seconds=4163, microseconds=94933), 'data': {'status': 'afk'}}


now that we have some rough bounds, let's examine the 

In [49]:
unfocused_durations=np.array([event.duration.total_seconds() for event in unfocused_events])
flow_durations=np.array([event.duration.total_seconds() for event in flow_events])

print(f'Average unfocused duration: {unfocused_durations.mean()}')
print(f'Average flow duration: {flow_durations.mean()}')

Average unfocused duration: 98.31573163157894
Average flow duration: 25.785870621621623
